<a href="https://colab.research.google.com/github/edgarMuno/TFM/blob/main/TFMClasificacitioEvent_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
# 🔹 Verificar si hay GPU disponible
print("✅ Dispositivos disponibles:", tf.config.list_physical_devices())

# 🔹 Forzar uso de la GPU en TensorFlow
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)  # Evita uso excesivo de RAM
        print("✅ GPU configurada correctamente.")
    except RuntimeError as e:
        print(e)
else:
    print("⚠️ No se detectó GPU. TensorFlow usará CPU.")


✅ Dispositivos disponibles: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
✅ GPU configurada correctamente.


se quito la cetorgira normal

In [3]:
# 📂 Rutas de los archivos preprocesados
train_path = "/content/drive/MyDrive/TFM/Train_process"
test_path = "/content/drive/MyDrive/TFM/Test_process"

# 🔹 Hiperparámetros
SEED = 12
IMG_HEIGHT = 64
IMG_WIDTH = 64
BATCH_SIZE = 32
EPOCHS = 4
LR =  0.00003
NUM_CLASSES = 13
CLASS_LABELS = ['Abuse','Arrest','Arson','Assault','Burglary','Explosion','Fighting',
                'RoadAccidents','Robbery','Shooting','Shoplifting','Stealing','Vandalism']


In [4]:
def npz_generator(folder_path):
    """Generador que carga los datos desde los archivos .npz"""
    class_files = sorted(os.listdir(folder_path))

    for file in class_files:
        file_path = os.path.join(folder_path, file)
        data = np.load(file_path, allow_pickle=True)
        X, y = data["X"], data["y"]

        # 🔹 Convertir imágenes a 3 canales
        X = np.repeat(X, 3, axis=-1)  # (64, 64, 1) → (64, 64, 3)

        # 🔹 Convertir etiquetas a números
        class_to_index = {class_name: idx for idx, class_name in enumerate(CLASS_LABELS)}
        y = np.array([class_to_index[label] for label in y])  # Convertir etiquetas a índice

        # 🔹 One-hot encoding
        y = tf.keras.utils.to_categorical(y, num_classes=NUM_CLASSES)

        for i in range(len(X)):  # Iterar sobre cada imagen
            yield X[i], y[i]  # Retornar una imagen a la vez


In [5]:
train_dataset = tf.data.Dataset.from_generator(
    lambda: npz_generator(train_path),
    output_signature=(
        tf.TensorSpec(shape=(IMG_HEIGHT, IMG_WIDTH, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(NUM_CLASSES,), dtype=tf.float32)
    )
).shuffle(buffer_size=1024).batch(BATCH_SIZE).repeat().prefetch(1)

test_dataset = tf.data.Dataset.from_generator(
    lambda: npz_generator(test_path),
    output_signature=(
        tf.TensorSpec(shape=(IMG_HEIGHT, IMG_WIDTH, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(NUM_CLASSES,), dtype=tf.float32)
    )
).shuffle(buffer_size=1024).batch(BATCH_SIZE).repeat().prefetch(1)


In [6]:
# 📂 Contar el total de imágenes en entrenamiento y validación
total_train_images = sum([np.load(os.path.join(train_path, file))["X"].shape[0] for file in sorted(os.listdir(train_path))])
total_test_images = sum([np.load(os.path.join(test_path, file))["X"].shape[0] for file in sorted(os.listdir(test_path))])

# 🔹 Calcular los steps por epoch asegurando que no sean mayores a los datos disponibles
steps_per_epoch = max(1, total_train_images // BATCH_SIZE)
validation_steps = max(1, total_test_images // BATCH_SIZE)

print(f"✅ Steps per epoch: {steps_per_epoch}, Validation steps: {validation_steps}")


✅ Steps per epoch: 4977, Validation steps: 724


In [8]:
print(f"✅ Total de imágenes en entrenamiento: {total_train_images}")
print(f"✅ Total de imágenes en validación: {total_test_images}")
print(f"✅ Steps per epoch (batches de {BATCH_SIZE}): {steps_per_epoch}")
print(f"✅ Validation steps (batches de {BATCH_SIZE}): {validation_steps}")


✅ Total de imágenes en entrenamiento: 318577
✅ Total de imágenes en validación: 46356
✅ Steps per epoch (batches de 64): 4977
✅ Validation steps (batches de 64): 724


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
def feature_extractor(inputs):
    base_model = tf.keras.applications.DenseNet121(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
                                                   include_top=False,
                                                   weights="imagenet")(inputs)
    return base_model

def classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(NUM_CLASSES, activation="softmax", name="classification")(x)

    return x

def define_compile_model():
    inputs = tf.keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    classification_output = classifier(feature_extractor(inputs))
    model = tf.keras.Model(inputs=inputs, outputs=classification_output)

    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=LR),
                loss='categorical_crossentropy',
                metrics=[tf.keras.metrics.AUC()])

    return model

# 🔹 Crear el modelo
model = define_compile_model()
model.summary()


29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 64, 64, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ densenet121 (Functional)             │ (None, 2, 2, 1024)          │       7,037,504 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1024)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         262,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1024)                │         263,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ classification (Dense)               │ (None, 13)                  │           6,669 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,094,541 (30.88 MB)

 Trainable params: 8,010,893 (30.56 MB)

 Non-trainable params: 83,648 (326.75 KB)

history = model.fit(
    train_subset,
    validation_data=test_subset,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,  # 🔹 Ahora usamos los valores correctos
    validation_steps=validation_steps
)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[early_stopping]
)


Epoch 1/4
1869/4977 ━━━━━━━━━━━━━━━━━━━━ 4:17 83ms/step - auc: 0.6708 - loss: 2.7434